In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

df = 'https://axe.inline-ltd.ru/data/meatinfo.csv'
df = pd.read_csv(df, delimiter=';', encoding='utf-8')

#Виды продукции, для которых в датасете есть не менее 500 примеров
min_examples = 500
selected_products = df['mtype'].value_counts()[df['mtype'].value_counts() >= min_examples].index.tolist()
df = df[df['mtype'].isin(selected_products)]

#Разделение на обучающую (80) и тестовую (20) выборки
X, y = train_test_split(df, test_size=0.2, random_state=42)

#text to tensor PyTorch
class TextDataset(Dataset):
    def __init__(self, texts, labels, vocab):
        self.texts = texts
        self.labels = labels
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)
        self.vocab = vocab

    def tokenize_text(self, text):
        tokens = text.split()
        return [self.vocab.index(token) if token in self.vocab else self.vocab.index('<UNK>') for token in tokens]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized_text = self.tokenize_text(text)
        return tokenized_text, label

    def get_label_name(self, label):
        return self.label_encoder.inverse_transform([label])[0]

#Модель
class TextClassifier(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        pooled = embedded.mean(1)
        out = torch.relu(self.fc1(pooled))
        out = self.fc2(out)
        return out

def train_model(model, train_loader, criterion, optimizer):
    model.train()
    epoch_loss = 0
    for texts, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(train_loader)

def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for texts, labels in test_loader:
            outputs = model(texts)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

input_dim = len(vocab)  # Размер словаря
embedding_dim = 100
hidden_dim = 128
output_dim = len(selected_products)
num_epochs = 50
batch_size = 64
learning_rate = 3e-4

all_words = ' '.join(X['text']).split()
word_counter = Counter(all_words)
vocab = ['<PAD>', '<UNK>'] + [word for word, count in word_counter.items() if count > 1]

train_dataset = TextDataset(X['text'].values, X['mtype'].values, vocab)
test_dataset = TextDataset(y['text'].values, y['mtype'].values, vocab)

def collate_fn(batch):
    texts = [torch.LongTensor(item[0]) for item in batch]
    labels = torch.LongTensor([item[1] for item in batch])
    texts = nn.utils.rnn.pad_sequence(texts, batch_first=True, padding_value=0)
    return texts, labels

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

model = TextClassifier(input_dim, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, criterion, optimizer)
    test_accuracy = evaluate_model(model, test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/50, Train Loss: 1.4458, Test Accuracy: 0.5176
Epoch 2/50, Train Loss: 1.4227, Test Accuracy: 0.5192
Epoch 3/50, Train Loss: 1.3971, Test Accuracy: 0.5189
Epoch 4/50, Train Loss: 1.3498, Test Accuracy: 0.5237
Epoch 5/50, Train Loss: 1.2682, Test Accuracy: 0.5532
Epoch 6/50, Train Loss: 1.1819, Test Accuracy: 0.5912
Epoch 7/50, Train Loss: 1.0961, Test Accuracy: 0.5915
Epoch 8/50, Train Loss: 1.0198, Test Accuracy: 0.6022
Epoch 9/50, Train Loss: 0.9442, Test Accuracy: 0.6724
Epoch 10/50, Train Loss: 0.8665, Test Accuracy: 0.6974
Epoch 11/50, Train Loss: 0.8021, Test Accuracy: 0.7172
Epoch 12/50, Train Loss: 0.7537, Test Accuracy: 0.6971
Epoch 13/50, Train Loss: 0.6899, Test Accuracy: 0.7172
Epoch 14/50, Train Loss: 0.6477, Test Accuracy: 0.7628
Epoch 15/50, Train Loss: 0.6068, Test Accuracy: 0.7749
Epoch 16/50, Train Loss: 0.5736, Test Accuracy: 0.7661
Epoch 17/50, Train Loss: 0.5421, Test Accuracy: 0.7838
Epoch 18/50, Train Loss: 0.5020, Test Accuracy: 0.8078
Epoch 19/50, Train 

In [ ]:
samples = [
    "Говядина блочная 2 сорт в наличии ООО 'АгроСоюз' реализует блочную говядину 2 сорт (80/20)",
    "Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.",
    "Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:",
    "Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.",
    "Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг.",
    "Мясо премиум Предлагаем котлетное мясо мраморной говядины.",
    "спинка цб"
]

tokenized_samples = [train_dataset.tokenize_text(sample) for sample in samples]
tensor_samples = [torch.LongTensor(sample) for sample in tokenized_samples]

padded_samples = nn.utils.rnn.pad_sequence(tensor_samples, batch_first=True, padding_value=0)

with torch.no_grad():
    model.eval()
    outputs = model(padded_samples)
    _, predicted_labels = torch.max(outputs, 1)

predicted_labels = predicted_labels.tolist()
predicted_product_names = [train_dataset.get_label_name(label) for label in predicted_labels]

for sample, predicted_product_name in zip(samples, predicted_product_names):
    print(f"Текст: {sample}")
    print(f"Предсказание: {predicted_product_name}")
    print()

Текст: Говядина блочная 2 сорт в наличии ООО 'АгроСоюз' реализует блочную говядину 2 сорт (80/20)
Предсказание: Говядина

Текст: Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.
Предсказание: Говядина

Текст: Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:
Предсказание: Кура

Текст: Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.
Предсказание: Говядина

Текст: Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг.
Предсказание: Свинина

Текст: Мясо премиум Предлагаем котлетное мясо мраморной говядины.
Предсказание: Говядина

Текст: спинка цб
Предсказание: Цыпленок

